<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

Cloning into 'tol403-lokaverkefni'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 3), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 491.96 KiB | 2.51 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [26]:
class Node:
  def __init__(self, osmid, x, y, primary):
    self.osmid = int(osmid)
    self.x = float(x)
    self.y = float(y)
    self.primary = bool(primary)

  def __repr__(self):
        return f"Node(id={self.osmid}, x={self.x}, y={self.y}, primary={self.primary})"

class Edge:
  def __init__(self, u, v, length, name):
    self.u = int(u)
    self.v = int(v)
    self.length = float(length)
    self.name = name

  def __repr__(self):
        return f"Edge(u={self.u}, v={self.v}, length={self.length}, name={self.name})"

class Graph:
    def __init__(self):
        self.adjacency_list = {}

    def add_node(self, node):
        if node.osmid not in self.adjacency_list:
            self.adjacency_list[node.osmid] = []

    def add_edge(self, edge):
        u = edge.u
        v = edge.v
        weight = edge.length
        if u not in self.adjacency_list:
            self.add_node(Node(osmid=u, x=0, y=0, primary=False))
        if v not in self.adjacency_list:
            self.add_node(Node(osmid=v, x=0, y=0, primary=False))
        self.adjacency_list[u].append((v, weight))

    def get_neighbors(self, node_id):
        return self.adjacency_list.get(node_id, [])

In [27]:
# 2.3.1
import csv

def load_csv_into_list(filepath, cls):
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file, delimiter = "\t")
        objects = []
        for row in reader:
            obj = cls(
                **{k: v for k, v in row.items()}
            )
            objects.append(obj)
        return objects

nodes = load_csv_into_list('tol403-lokaverkefni/data/nodes.tsv', Node)
edges = load_csv_into_list('tol403-lokaverkefni/data/edges.tsv', Edge)

graph = Graph()

for node in nodes:
    graph.add_node(node)

for edge in edges:
    graph.add_edge(edge)

In [28]:
# 2.3.2
## Steps

### 1, Invert graph (edge 1->2 becoes 2->1)
### 2, Run Dijkstra for each starting point (yes this is slow af)
### 2.1, Initialise each point's distance to infinity, and its closest starting point to itself
### 2.2, For each node, see if this new path is shorter than the previous known path
###      If it is, then set this as the closest starting point

import heapq

def calculate_distances(graph, end_vertices):
    # Invert the graph
    inverted_graph = Graph()
    for u in graph.adjacency_list:
        for (v, weight) in graph.adjacency_list[u]:
            inverted_graph.add_edge(Edge(v, u, weight, ""))

    distances = {node: float('infinity') for node in inverted_graph.adjacency_list}
    closest_start = {node: None for node in inverted_graph.adjacency_list}

    heap = []

    for vertex in end_vertices:
        distances[vertex] = 0
        closest_start[vertex] = vertex
        heapq.heappush(heap, (0, vertex))

    # Dijkstra
    while heap:
        current_distance, u = heapq.heappop(heap)

        if current_distance > distances[u]:
            continue

        for (v, weight) in inverted_graph.adjacency_list.get(u, []):
            new_distance = current_distance + weight
            if distances[v] > new_distance:
                distances[v] = new_distance
                # Set closest end point
                closest_start[v] = closest_start[u]
                heapq.heappush(heap, (new_distance, v))

    return distances, closest_start

## Test stuff
distances,closest_start = calculate_distances(graph, [nodes[50].osmid, nodes[11].osmid])

end_nodes = sorted(distances.keys())

### Print first 50
for node in end_nodes[:50]:
    print(f"Node: {node}, Distance: {distances[node]}, Closest Start: {closest_start[node]}")


Node: 12885866, Distance: 27.729455385158538, Closest Start: 12886003
Node: 12885876, Distance: 1071.7628682338736, Closest Start: 12886003
Node: 12885922, Distance: 8778.166253656374, Closest Start: 12886003
Node: 12885923, Distance: 8.73743989282635, Closest Start: 12886003
Node: 12885924, Distance: 119.12594487251242, Closest Start: 12886003
Node: 12885930, Distance: 984.3324217994717, Closest Start: 12886003
Node: 12885937, Distance: 6015.709756684763, Closest Start: 12886003
Node: 12885952, Distance: 1055.6315964510645, Closest Start: 12886003
Node: 12885974, Distance: 104.45716130620653, Closest Start: 12886003
Node: 12885979, Distance: 1027.288130529451, Closest Start: 12886003
Node: 12885991, Distance: 1004.645467783603, Closest Start: 12886003
Node: 12886003, Distance: 0, Closest Start: 12886003
Node: 12886006, Distance: 41.91013883585239, Closest Start: 12886003
Node: 12886026, Distance: 74.87399349884423, Closest Start: 12886003
Node: 12886027, Distance: 1096.603437099763, C